In [1]:
from pyserver.model.mT5thai.mT5Thai import Mt5Thai
from pyserver.webScrap import webScrap
from IPython.core.display import display, HTML
import warnings
warnings.simplefilter("ignore", UserWarning)

In [2]:
paragraph = webScrap("https://www.thairath.co.th/news/local/northeast/2695085")

In [3]:
model = Mt5Thai()

thanathorn/mt5-cpe-kmutt-thai-sentence-sum


You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [4]:
model.tokenize(paragraph)
status, output_text, highlighted_text = model.summarize()

In [12]:
display(HTML(highlighted_text.replace("&lt;/s&gt;", "")))

In [6]:
display(HTML(output_text))

### NEWS

In [7]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

model_name="thanathorn/mt5-cpe-kmutt-thai-sentence-sum"
# model_name = "Nopphakorn/mt5-news-thaisum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name, 
                        output_attentions=True,
                        output_scores=True,
                        # from_pt=True,
                        output_hidden_states=True,
                    )

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [8]:
input_ids2 = tokenizer.encode(paragraph, return_tensors="pt", padding="longest")
min_length = 50
output2 = model.generate(
                        input_ids2, 
                        min_length=min_length,     # Parameters that control the length of the output
                        max_length=min_length*2, 
                        do_sample=True,                 # Parameters that control the generation strategy used
                        num_beams=3,    
                        output_attentions=True,         # Parameters that define the output variables of `generate`
                        output_scores=True, 
                        return_dict_in_generate=True,
                        output_hidden_states=True,
                        )
output_text2 = tokenizer.decode(output2.sequences[0], skip_special_tokens=True)
tokens2 = tokenizer.convert_ids_to_tokens(input_ids2[0])

In [9]:
import pandas as pd
import numpy as np
import html
import tensorflow as tf
def html_escape(text):
    return html.escape(text)

# Highlight & export function
def highlightedText(tokens, output, num_head=0, pad='', cut_off=0.5, save_html=False) -> list:
    '''
    https://adataanalyst.com/machine-learning/highlight-text-using-weights/
    '''
    highlighted_text = []
    output_att = tf.stack(output.encoder_attentions)

    reshaped_scores = tf.reduce_mean(output_att[-1,:,num_head], axis=0)
    df = pd.DataFrame(reshaped_scores).sum(axis=0)
    df = 1/df
    for i, word in enumerate(tokens):
        
        if '▁' in word: word = word.replace('▁', ' ') # change space '▁' to ' '

        # df_sumcol = pd.DataFrame( attention[encoder1][0][encoder2] ).sum(axis=0) # reduce dim(N, N) to dim(N, 1) by sum column
        df /= df.max() # normalize

        weight = df[i]
        if weight < cut_off: weight = 0
        
        highlighted_text.append('<span style="background-color:rgba(251,155,59,' + str(weight) + ');">' + html_escape(word) + '</span>')
        # highlighted_text.append('' + html_escape(word) + '')
        
    highlighted_text = pad.join(highlighted_text)

    display(HTML(highlighted_text))

    if save_html:
        html = HTML(highlighted_text).data
        # file name ex. attention_0_1.html
        with open(f'attention.html', 'w') as f:
            f.write(html)

    return highlighted_text

In [10]:
z = highlightedText(tokens2, output2, 11)

In [11]:
display(HTML(output_text))